In [1]:
import pandas as pd
import numpy as np

In [28]:
trip = pd.read_csv('data/tripP')
weather = pd.read_csv('data/weatherP')
station = pd.read_csv('data/stationP')

In [29]:
trip = trip.drop(['start_date', 'date', 'holiday'], axis=1)

In [30]:
def sepTrainTest(df):
    train = df.loc[:df.shape[0] - 50000]
    test = df.loc[-50000:]
    return extractxy(train), extractxy(test)
    
def extractxy(df):
    y = df.duration
    del df['duration']
    return df, y

In [31]:
train, test = sepTrainTest(trip)

In [32]:
from sklearn import ensemble
rfr = ensemble.RandomForestRegressor(
    n_jobs=-1,
    n_estimators=50,
    oob_score=True)

In [33]:
rfr.fit(train[0], train[1])

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=50, n_jobs=-1, oob_score=True, random_state=None,
           verbose=0, warm_start=False)

In [34]:
from sklearn.metrics import mean_squared_error
mean_squared_error(rfr.predict(test[0]), test[1])

75898535.514714837